In [2]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch 

In [3]:
import mysql.connector

In [4]:
import json

# 日期处理

In [5]:
def date_convert(dot_date_str):
    '''xxxx.xx.xx --> xxxx-xx-xx
    then pack into json for es
    '''
    y,m,d = dot_date_str.split('.')
    new_date = '{year}-{month}-{date}'.format(year=y,month=m,date=d)
    return new_date
#     return {new_date:{'type':'date','formate':'yyyy-mm-dd'}}

In [6]:
def date_autofill(dot_date_str):
    '''xxxx --> xxxx-01-01
    then pack into json for es
    '''
    new_date = '{year}-{month}-{date}'.format(year=dot_date_str,month='01',date='01')
    return new_date

In [7]:
def data_to_json(data_atom,columns):
    data = data_atom
    data_dict = {k:str(data[num]) for num,k in enumerate(columns)}
    return data_dict

# 读取weibo数据

In [8]:
comment_df = pd.read_pickle('../data/weibo_data/comment.pkl')
weibo_df = pd.read_pickle('../data/weibo_data/weibo.pkl')

In [10]:
weibo_df

,_id,comment_num,content,crawl_time,created_at,image_url,like_num,location,location_map_info,nick_name,origin_weibo_content,origin_weibo_url,repost_num,search_query,tool,user_id,weibo_url
0,1642591402_HCY6jcf7y,883,#李荣浩求婚钻戒# 7月11日，#李荣浩成功求婚杨丞琳# 后大方晒出钻戒和合照宣布喜讯。有台...,1562836636,2019-07-11 14:27,None,10869,None,None,新浪娱乐,None,None,434,#李荣浩成功求婚杨丞琳#,微博 weibo.com,1642591402,https://weibo.com/1642591402/HCY6jcf7y
1,1642591402_HCWAjce8C,3723,#李荣浩成功求婚杨丞琳# 刚刚，@李荣浩 在生日当天晒出与@楊丞琳 的合照宣布成功求婚的喜讯...,1562836636,2019-07-11 10:36,None,54236,None,None,新浪娱乐,None,None,2818,#李荣浩成功求婚杨丞琳#,微博 weibo.com,1642591402,https://weibo.com/1642591402/HCWAjce8C
2,1878335471_HCWB55c1j,880,#李荣浩成功求婚杨丞琳##杨丞琳李荣浩 甜# 刚刚，@李荣浩 在生日当天晒出与@楊丞琳 的合...,1562836636,2019-07-11 10:38,None,21145,None,None,新浪综艺,None,None,784,#李荣浩成功求婚杨丞琳#,微博 weibo.com,1878335471,https://weibo.com/1878335471/HCWB55c1j
3,3927390636_HCYmklSOL,929,#李荣浩求婚钻戒# 目测约2克拉，约合人民币66~88万元！我好酸，钻戒比李老师眼镜还大[酸...,1562836636,2019-07-11 15:07,None,7179,None,None,娱乐报姐,None,None,324,#李荣浩成功求婚杨丞琳#,李荣浩超话,3927390636,https://weibo.com/3927390636/HCYmklSOL
4,1952225710_HCWDH1AUC,493,#李荣浩成功求婚杨丞琳#李荣浩刚刚在微博晒出与杨丞琳的合照，宣布求婚成功的喜讯！恭喜两位有情...,1562836636,2019-07-11 10:44,None,3463,None,None,镰刀刮腋毛,None,None,272,#李荣浩成功求婚杨丞琳#,None,1952225710,https://weibo.com/1952225710/HCWDH1AUC
5,2115034114_HCWDfxEnn,2113,#李荣浩成功求婚杨丞琳#今日份的狗粮来了，每日一酸成就达成！有情人终成眷属，最喜欢的一对。恭...,1562836636,2019-07-11 10:43,None,17086,None,None,娱乐明星团,None,None,767,#李荣浩成功求婚杨丞琳#,微博 weibo.com,2115034114,https://weibo.com/2115034114/HCWDfxEnn
6,1642591402_HCWSuwNiY,538,#李荣浩杨丞琳回应求婚# 11日，#李荣浩成功求婚杨丞琳# ，随后@楊丞琳 向新浪娱乐独家回...,1562836636,2019-07-11 11:21,None,10245,None,None,新浪娱乐,None,None,263,#李荣浩成功求婚杨丞琳#,微博 weibo.com,1642591402,https://weibo.com/1642591402/HCWSuwNiY
7,1266269835_HCWNMyMXc,82,#李荣浩成功求婚杨丞琳# #李荣浩杨丞琳回应求婚# 今天，@李荣浩 微博透露求婚@楊丞琳 成...,1562836636,2019-07-11 11:09,None,1980,None,None,新浪音乐,None,None,58,#李荣浩成功求婚杨丞琳#,微博 weibo.com,1266269835,https://weibo.com/1266269835/HCWNMyMXc
8,1625300327_HCWYL7xvT,345,#李荣浩成功求婚杨丞琳#刚吃完生日祝福狗粮，又来了求婚成功大糖！今天真是个好日子，恭喜李荣浩...,1562836639,2019-07-11 11:36,None,388,None,None,综艺小可爱,None,None,322,#李荣浩成功求婚杨丞琳#,None,1625300327,https://weibo.com/1625300327/HCWYL7xvT
9,5092918149_HCWFPEAtV,324,#李荣浩成功求婚杨丞琳# 李荣浩羞谈与杨丞琳相识经历，爱得坦荡强吻杨丞琳“我就忍不住了”...,1562836639,2019-07-11 10:49,None,1878,None,None,一介娱夫,None,None,235,#李荣浩成功求婚杨丞琳#,360安全浏览器,5092918149,https://weibo.com/5092918149/HCWFPEAtV


In [14]:
comment_columns = comment_df.columns.tolist()

In [15]:
comment_columns = ['w_id'] + comment_columns[1:]

In [16]:
comment_columns

['w_id',
 'comment_user_id',
 'comment_user_nick_name',
 'content',
 'crawl_time',
 'created_at',
 'like_num',
 'weibo_url']

In [58]:
weibo_columns = weibo_df.columns.tolist()

In [59]:
weibo_columns = ['w_id'] + weibo_columns[1:]

# 创建ES对象

In [31]:
from elasticsearch.helpers import bulk

from elasticsearch.helpers import bulk

es = Elasticsearch(hosts='10.8.128.205',port=49200,timeout=30, max_retries=10, retry_on_timeout=True)

# 批量插入comment

In [43]:
sample_item = comment_df.iloc[num]
sample_dict = data_to_json(sample_item,comment_columns)
es.create(index='weibo-comment-index',id='test',body=json.dumps(sample_dict))

{'_index': 'weibo-comment-index',
 '_type': '_doc',
 '_id': 'test',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [45]:
counter = 0
index_batch = []
batch = 10000
for num,_ in enumerate(comment_df.itertuples()):
    if num %batch==0:
        print('[{0}] Num:{1}'.format(time.ctime(),num))
    sample_item = comment_df.iloc[num]
    sample_dict = data_to_json(sample_item,comment_columns)
    if counter <batch:
        index_batch.append(sample_dict)
        counter += 1
    else:
        index_batch.append(sample_dict)
        print('Bulking....')
        time1 = time.time()
        bulk(es,index_batch,index='weibo-comment-index',raise_on_error=True)
        time2 = time.time()
        print('Total time:{0:.2f}min'.format((time2-time1)/60))
        counter = 0
        index_batch = []
bulk(es,index_batch,index='weibo-comment-index',raise_on_error=True)
es.indices.refresh(index="weibo-comment-index")

[Thu Jul 18 16:31:26 2019] Num:0
[Thu Jul 18 16:31:30 2019] Num:10000
Bulking....
Total time:0.07min
[Thu Jul 18 16:31:38 2019] Num:20000
Bulking....
Total time:0.06min
[Thu Jul 18 16:31:45 2019] Num:30000
Bulking....
Total time:0.06min
[Thu Jul 18 16:31:53 2019] Num:40000
Bulking....
Total time:0.06min
[Thu Jul 18 16:32:01 2019] Num:50000
Bulking....
Total time:0.05min
[Thu Jul 18 16:32:08 2019] Num:60000
Bulking....
Total time:0.06min
[Thu Jul 18 16:32:16 2019] Num:70000
Bulking....
Total time:0.06min
[Thu Jul 18 16:32:24 2019] Num:80000
Bulking....
Total time:0.05min


{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

# 批量插入weibo

In [63]:
sample_item = weibo_df.iloc[0]
sample_dict = data_to_json(sample_item,weibo_columns)

In [65]:
es.create(index='weibo-tweet-index',id='test',body=json.dumps(sample_dict))

{'_index': 'weibo-tweet-index',
 '_type': '_doc',
 '_id': 'test',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [66]:
counter = 0
index_batch = []
batch = 10000
for num,_ in enumerate(weibo_df.itertuples()):
    if num %batch==0:
        print('[{0}] Num:{1}'.format(time.ctime(),num))
    sample_item = weibo_df.iloc[num]
    sample_dict = data_to_json(sample_item,comment_columns)
    if counter <batch:
        index_batch.append(sample_dict)
        counter += 1
    else:
        index_batch.append(sample_dict)
        print('Bulking....')
        time1 = time.time()
        bulk(es,index_batch,index='weibo-tweet-index',raise_on_error=True)
        time2 = time.time()
        print('Total time:{0:.2f}min'.format((time2-time1)/60))
        counter = 0
        index_batch = []
bulk(es,index_batch,index='weibo-tweet-index',raise_on_error=True)
es.indices.refresh(index="weibo-tweet-index")

[Thu Jul 18 16:42:17 2019] Num:0


{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}